##NPPE Speaker Classification Instructions
1. Install Required Packages
Ensure Python is installed and run the following command to install the necessary libraries:


`pip install transformers==4.44.2 datasets==3.1.0 torch==2.5.0`

2. Set Up Environment
Verify the availability of a GPU for faster processing. Use CUDA (GPU support) if available by transferring your PyTorch models and data for feature extraction and CNN training to the available device.

3. Loading and Processing Data
Dataset Loading: Load the VoxPopuli dataset using the
1
datasets
library with
1
streaming=True
for memory efficiency. Load the dataset with language "slovene" and split "train."
Random Shuffling: Shuffle the dataset using a random seed of
1
42
. Also, fix the same seed for
1
torch.manual_seed
.
4. Feature Extraction Using Wav2Vec2
Loading Model and Processor: Load the Wav2Vec2 model and processor from Hugging Face’s transformers library to extract audio features. Use the pretrained model
1
facebook/wav2vec2-base-960h
.
Feature Extraction Function: Define a function to process each audio sample and extract its feature representation. Apply this function to obtain Wav2Vec2 embeddings for the dataset.
5. Data Filtering and Label Mapping
Filtering Short Samples: Exclude samples with fewer than 200 frames.
Label Mapping: Assign each unique speaker an integer label. Sort the speaker list using the
1
sorted
function before mapping and store mappings in a dictionary.
6. Model Definition
Define a custom CNN model with the following architecture:

Conv1: In channels: 768, Out channels: 256, Kernel size: 3
BatchNorm1: Features: 256
ReLU1: Applied after BatchNorm1
MaxPool1: Kernel size: 2
Conv2: In channels: 256, Out channels: 128, Kernel size: 3
BatchNorm2: Features: 128
ReLU2: Applied after BatchNorm2
MaxPool2: Kernel size: 2
Conv3: In channels: 128, Out channels: 32, Kernel size: 3
BatchNorm3: Features: 32
ReLU3: Applied after BatchNorm3
MaxPool3: Kernel size: 2
Global Average Pooling: Averaged across the temporal dimension
FC1: Input features: 32, Output features: 128
ReLU4: Applied after FC1
FC2: Input features: 128, Output features: Number of classes **bold text**
7. Data Splitting and DataLoader
Data Splitting: Split the data into training, validation, and test sets using sklearn. Divide the dataset by taking 80% for train and 20% for test. Then take another 10% of the train data for valid set. Set random state to 42 and use sklearn for this stage.
DataLoader: Use PyTorch’s DataLoader with a custom collate function to convert features and labels to tensors, truncating features to the first 200 frames. Batch sizes:
Train: 100
Validation: 10
Test: 1
8. Training the Model
Loss and Optimizer: Use cross-entropy loss and Adam optimizer with a learning rate of
1
0.001
.
Training Loop: Train for 100 epochs. For each epoch:
Set the model to training mode.
Zero the optimizer’s gradient.
For each batch, forward propagate, calculate loss, and backpropagate.
Track average loss for the epoch.
Validation: Evaluate on the validation set after each epoch by calculating loss without weight updates. Track validation loss to monitor performance.
9. Evaluating on the Test Set
Set the model to evaluation mode. Compute class predictions for each test sample, calculate accuracy using
1
sklearn
's
1
accuracy_score
, and display the test accuracy.

In [ ]:
!pip install transformers==4.44.2 datasets==3.1.0 torch==2.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 132.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from datasets import load_dataset, get_dataset_config_names

In [ ]:
get_dataset_config_names("facebook/voxpopuli")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

voxpopuli.py:   0%|          | 0.00/8.84k [00:00<?, ?B/s]

The repository for facebook/voxpopuli contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/facebook/voxpopuli.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


['en',
 'de',
 'fr',
 'es',
 'pl',
 'it',
 'ro',
 'hu',
 'cs',
 'nl',
 'fi',
 'hr',
 'sk',
 'sl',
 'et',
 'lt',
 'en_accented',
 'multilang']

In [ ]:
dataset = load_dataset("facebook/voxpopuli", "sl", split="train", streaming=True)

In [ ]:
for i in dataset:
  print(i)
  break

{'audio_id': '20140417-0900-PLENARY-7-sl_20140417-10:42:28_17', 'language': 13, 'audio': {'path': 'train_part_0/20140417-0900-PLENARY-7-sl_20140417-10:42:28_17.wav', 'array': array([ 2.44140625e-04, -1.22070312e-04, -6.10351562e-05, ...,
       -7.01904297e-04,  0.00000000e+00,  2.44140625e-04]), 'sampling_rate': 16000}, 'raw_text': 'V tem duhu se zahvaljujem poročevalcem v senci in vodji delegacije za Japonsko gospodu van Baalenu, kot tudi njenim posameznim članom za sodelovanje in koristne konzultacije.', 'normalized_text': 'v tem duhu se zahvaljujem poročevalcem v senci in vodji delegacije za japonsko gospodu van baalenu kot tudi njenim posameznim članom za sodelovanje in koristne konzultacije.', 'gender': 'male', 'speaker_id': '23693', 'is_gold_transcript': True, 'accent': 'None'}


In [ ]:
# c = 0
# for i in dataset:
#   c += 1
# c

In [ ]:

dataset = dataset.shuffle(seed=42)

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
from tqdm import tqdm
# Load the Wav2Vec2 processor and model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch

# Move model to GPU
model = model.to('cuda')
min_frame = float('inf')
max_frame = 0
count = 0
def process_audio(data):
    global count, min_frame, max_frame
    count += 1
    # print(count)

    # Move input data to GPU
    inputs = processor(data["audio"]["array"], sampling_rate=16000, return_tensors="pt", padding=True)
    inputs = {key: val.to('cuda') for key, val in inputs.items()}  # Move inputs to GPU

    with torch.no_grad():
        # Forward pass on GPU
        outputs = model(**inputs)

    # Move the output back to CPU
    features = outputs.last_hidden_state.squeeze()
    frame = outputs.last_hidden_state.cpu().numpy().shape[1]
    if frame < min_frame:
      min_frame = frame
    if frame > max_frame:
      max_frame = frame
    # print(frame)
    return {'features': features, 'label': data['speaker_id'], 'frame_rate': frame}


In [ ]:
x = process_audio(next(iter(dataset)))

In [ ]:
x['features'].shape, x['label'], x['frame_rate']

(torch.Size([324, 768]), '97019', 324)

In [ ]:
# Map the function over the dataset
processed_dataset = dataset.map(process_audio).filter(lambda x: x['frame_rate']>=200)

In [ ]:
unique_spk = dict()
spk_to_id = dict()
cur_spk = 0

c = 0
data_list = list()
for data in processed_dataset:
  c += 1
  if c == 1:
    print(data)
  if unique_spk.get(data['label']) is None:
    unique_spk[data['label']] = 1
    spk_to_id[data['label']] = cur_spk
    cur_spk += 1
  else:
    unique_spk[data['label']] += 1
  data['label'] = spk_to_id[data['label']]
  data['features'] = data['features'][:200].cpu().numpy()
  data_list.append(data)

c

{'audio_id': '20160706-0900-PLENARY-16-sl_20160706-17:46:12_5', 'language': 13, 'audio': {'path': 'train_part_0/20160706-0900-PLENARY-16-sl_20160706-17:46:12_5.wav', 'array': array([-0.00079346,  0.00195312,  0.00448608, ..., -0.13134766,
       -0.14324951, -0.10733032]), 'sampling_rate': 16000}, 'raw_text': 'Zato bom to svojo minuto posvetil razstavi o oljki, simbolu miru, preživetja in sožitja.', 'normalized_text': 'zato bom to svojo minuto posvetil razstavi o oljki simbolu miru preživetja in sožitja.', 'gender': 'male', 'speaker_id': '97019', 'is_gold_transcript': True, 'accent': 'None', 'features': tensor([[-0.2237, -0.0537,  0.2011,  ..., -0.0044,  0.0124, -0.0896],
        [-0.1868, -0.0021, -0.0329,  ...,  0.0359, -0.1378, -0.0867],
        [-0.2168,  0.1737, -0.2272,  ..., -0.0951,  0.0240,  0.1956],
        ...,
        [-0.0827, -0.1002,  0.4105,  ..., -0.0068,  0.0306, -0.1201],
        [-0.0823, -0.0750,  0.3703,  ..., -0.1898,  0.0454, -0.1828],
        [-0.0546, -0.0293,

1937

In [ ]:
print(f"  min_frame: {min_frame}")
print(f"  max_frame: {max_frame}")

  min_frame: 27
  max_frame: 1948


In [ ]:
len(data_list)

1937

In [ ]:
len(unique_spk)

12

In [ ]:
# Sort the list by the 'label' key
data_list = sorted(data_list, key=lambda x: x["label"])

In [ ]:
torch.manual_seed(42)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
# Set the random seed for reproducibility
torch.manual_seed(42)
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        # torch.manual_seed = 42
        # Conv1 -> BatchNorm1 -> ReLU1 -> MaxPool1
        self.conv1 = nn.Conv1d(in_channels=768, out_channels=256, kernel_size=3)
        self.bn1 = nn.BatchNorm1d(num_features=256)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)

        # Conv2 -> BatchNorm2 -> ReLU2 -> MaxPool2
        self.conv2 = nn.Conv1d(in_channels=256, out_channels=128, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(num_features=128)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2)

        # Conv3 -> BatchNorm3 -> ReLU3 -> MaxPool3
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=32, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(num_features=32)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool1d(kernel_size=2)

        # Global Average Pooling
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)  # Outputs a tensor of shape [batch_size, channels, 1]

        # Fully Connected Layers
        self.fc1 = nn.Linear(in_features=32, out_features=128)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(in_features=128, out_features=num_classes)

    def forward(self, x):
        # Conv1 -> BatchNorm1 -> ReLU1 -> MaxPool1
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        # Conv2 -> BatchNorm2 -> ReLU2 -> MaxPool2
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        # Conv3 -> BatchNorm3 -> ReLU3 -> MaxPool3
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.pool3(x)

        # Global Average Pooling
        x = self.global_avg_pool(x)  # Shape: [batch_size, channels, 1]
        x = x.squeeze(-1)  # Remove the last dimension, Shape: [batch_size, channels]

        # Fully Connected Layers
        x = self.fc1(x)
        x = self.relu4(x)
        x = self.fc2(x)

        return x

In [ ]:
for sample in samples_frame_200:
    sample['label'] = label_mapping.get(sample['label'], -1)  # Default to -1 if not found

# Check the updated samples
print(samples_frame_200)

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data_list, test_size=0.2, random_state=42)

# Further split train data into training (90%) and validation (10%)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)


In [ ]:
len(train_data)

1394

In [ ]:
from torch.utils.data import DataLoader, Dataset
# Step 2: Custom Dataset Class
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        features = item["features"]  # Assuming features are stored as numpy arrays
        label = item["label"]
        return torch.tensor(features, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Step 4: Create DataLoaders
batch_sizes = {"train": 100, "validation": 10, "test": 1}

train_dataset = CustomDataset(train_data)
val_dataset = CustomDataset(val_data)
test_dataset = CustomDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=batch_sizes["train"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_sizes["validation"], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_sizes["test"], shuffle=False)

# Example: Iterate through the train DataLoader
for batch_features, batch_labels in train_loader:
    print(f"Batch features shape: {batch_features.shape}, Batch labels shape: {batch_labels.shape}")
    break

Batch features shape: torch.Size([100, 200, 768]), Batch labels shape: torch.Size([100])


In [ ]:
len(train_loader)

14

In [ ]:
for batch_features, batch_labels in tqdm(train_loader, desc="Training"):
  print(batch_features.shape, batch_labels.shape)
  break

Training:   0%|          | 0/14 [00:00<?, ?it/s]

torch.Size([100, 200, 768]) torch.Size([100])


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Assuming `model` is already defined and matches the architecture you shared
# Assuming `train_loader` and `val_loader` are DataLoaders created as described earlier
model = CustomCNN(num_classes=len(unique_spk))
# Step 1: Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 2: Training and Validation Loop
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:

train_loss_each = list()
val_loss_each = list()
for epoch in range(1, num_epochs + 1):
    # Set the model to training mode
    model.train()
    running_train_loss = 0.0

    print(f"Epoch {epoch}/{num_epochs}")
    print("-" * 30)

    # Training Loop
    for batch_features, batch_labels in tqdm(train_loader, desc="Training"):
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        outputs = model(batch_features.transpose(1, 2).float())

        # Zero the optimizer gradients
        optimizer.zero_grad()

        # Forward pass
        # Compute the loss
        loss = criterion(outputs, batch_labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Accumulate the loss
        running_train_loss += loss.item()

    # Average training loss for the epoch
    avg_train_loss = running_train_loss / len(train_loader)
    train_loss_each.append(avg_train_loss)
    print(f"Training Loss: {avg_train_loss:.4f}")

    # Validation Loop
    model.eval()
    running_val_loss = 0.0

    with torch.no_grad():
        for val_features, val_labels in tqdm(val_loader, desc="Validation"):
            val_features, val_labels = val_features.to(device), val_labels.to(device)

            # Forward pass
            val_outputs = model(val_features.transpose(1, 2).float())

            # Compute the validation loss
            val_loss = criterion(val_outputs, val_labels)

            # Accumulate validation loss
            running_val_loss += val_loss.item()

    # Average validation loss for the epoch
    avg_val_loss = running_val_loss / len(val_loader)
    val_loss_each.append(avg_val_loss)
    print(f"Validation Loss: {avg_val_loss:.4f}\n")

# Save the trained model
torch.save(model.state_dict(), "cnn_model.pth")
print("Training complete. Model saved to cnn_model.pth")


Epoch 1/100
------------------------------


Training: 100%|██████████| 14/14 [00:01<00:00,  9.28it/s]


Training Loss: 2.2512


Validation: 100%|██████████| 16/16 [00:00<00:00, 134.59it/s]


Validation Loss: 2.2606

Epoch 2/100
------------------------------


...

Validation Loss: 0.9531

Epoch 100/100
------------------------------


Training: 100%|██████████| 14/14 [00:01<00:00,  9.28it/s]


Training Loss: 0.0003


Validation: 100%|██████████| 16/16 [00:00<00:00, 134.63it/s]

Validation Loss: 0.9509

Training complete. Model saved to cnn_model.pth


In [ ]:
import numpy as np
val_out = np.array([])
val_label = np.array([])
with torch.no_grad():
  for val_features, val_labels in tqdm(val_loader, desc="Validation"):
    val_features, val_labels = val_features.to(device), val_labels.to(device)
    val_label = np.append(val_label, val_labels.cpu().numpy())
    # Forward pass
    val_outputs = model(val_features.transpose(1, 2).float())
    out = torch.argmax(val_outputs, dim=1)

    val_out = np.append(val_out, out.cpu().numpy())

Validation: 100%|██████████| 16/16 [00:00<00:00, 126.45it/s]


In [ ]:
val_out.shape, val_label.shape

((155,), (155,))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(val_out, val_label)

0.7161290322580646

# Test

In [ ]:
import numpy as np
test_out = np.array([])
test_label = np.array([])
with torch.no_grad():
  for test_features, test_labels in tqdm(test_loader, desc="test"):
    test_features, test_labels = test_features.to(device), test_labels.to(device)
    test_label = np.append(test_label, test_labels.cpu().numpy())
    # Forward pass
    test_outputs = model(test_features.transpose(1, 2).float())
    out = torch.argmax(test_outputs, dim=1)

    test_out = np.append(test_out, out.cpu().numpy())




test: 100%|██████████| 388/388 [00:00<00:00, 588.07it/s]


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_out, test_label)

0.7474226804123711